In [ ]:
import ds
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sb
from matplotlib.colors import ListedColormap
import math

d_clean = pd.read_csv("Cleaned_DF.csv")

In [ ]:
features = ['CryoSleep','Age','VIP','RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Deck','Num','Side','route']

In [ ]:
data_exploration_tree = ds.divide_frame(dataf = d_clean[d_clean['route']>3.5],columns = features,teacher_column = "Transported",classifier = 'Tree', criterion_tree = "entropy", depth = 4, random_state_sample = 5,random_state_ppn = 2, fraction = 1)

3.5 < route
1519 CryoSleep == 1 ==> 1.0 [66,1453]

In [ ]:
mask1 = (d_clean['CryoSleep'] == 1)&(d_clean['route']>3.5)# ==> 1.0

In [ ]:
df_impure = d_clean[~mask1]
df_impure.shape

(6156, 14)

From here we try to find other reelations in impure frame

In [ ]:
def sigmoid(x): 
    return 1/(1+math.exp(-x))

In [ ]:
def z(ppn,x1,x2):
    return ppn.w_[0]+ ppn.w_[1]*x1 + ppn.w_[2]*x2

In [ ]:
features = ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck','Transported']
d1 = df_impure[features]

sb.pairplot(d1,hue = "Transported")

In [ ]:
corrMatrix = d1.corr()

sb.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
col_s = ['FoodCourt','Spa']
dz = d1[col_s+["Transported"]]

x1_mean = dz[col_s[0]].mean()
x1_sigma = dz[col_s[0]].std()

x2_mean = dz[col_s[1]].mean()
x2_sigma = dz[col_s[1]].std()

for i in dz.index:
    dz.loc[i,col_s[0]] = (dz.loc[i,col_s[0]] - x1_mean)/x1_sigma
    dz.loc[i,col_s[1]] = (dz.loc[i,col_s[1]] - x2_mean)/x2_sigma

In [ ]:
ppn = ds.divide_frame(dataf = dz,columns = col_s,teacher_column = "Transported",classifier = 'Perceptron', random_state_sample = 4,random_state_ppn = 3, fraction = 0.7, eta = 0.001, iter = 30,plot = True, res =0.01)

In [ ]:
ppn = ds.divide_frame(dataf = dz,columns = col_s,teacher_column = "Transported",classifier = 'SVC', kernel_ = 'rbf', gamma_ = 1, c=1, degree_ = 3, random_state_sample = 1,random_state_ppn = 1, fraction = 0.7, eta = 0.0001, iter = 300,plot = True, res =0.01)

In [ ]:
ppn = ds.divide_frame(dataf = dz,columns = col_s,teacher_column = "Transported",classifier = 'SVC', kernel_ = 'poly', gamma_ = 1, c=1, degree_ = 3, random_state_sample = 1,random_state_ppn = 1, fraction = 0.9,plot = True, res =0.01)

In [ ]:
ppn = ds.divide_frame(dataf = dz,columns = col_s,teacher_column = "Transported",classifier = 'SVC', kernel_ = 'linear', gamma_ = 1, c=2, random_state_sample = 2,random_state_ppn = 3, fraction = 0.7,plot = True, res =0.01)

In [ ]:
ppn = ds.divide_frame(dataf = dz,columns = col_s,teacher_column = "Transported",classifier = 'KNN', dimension = 2, neighbors =2, random_state_sample = 1,random_state_ppn = 1, fraction = 0.9, plot = True, res =0.01)

In [ ]:
ppn = ds.divide_frame(dataf = dz,columns = col_s,teacher_column = "Transported",classifier = 'Tree', criterion_tree = "entropy", depth = 5, random_state_sample = 1,random_state_ppn = 2, fraction = 0.8,plot = True, res = 0.01)

In [ ]:
ppn = ds.divide_frame(dataf = dz,columns = col_s,teacher_column = "Transported",classifier = 'Adaline', random_state_sample = 2,random_state_ppn = 3, fraction = 0.7, eta = 0.0001, iter = 150,plot = True, res = 0.01)

In [ ]:
for i in d1.index:
    d1.loc[i,'f1'] = sigmoid(z(ppn,d1.loc[i,col_s[0]],d1.loc[i,col_s[1]]))

In [ ]:
corrMatrix = d1.corr()

sb.heatmap(corrMatrix, annot=True)
plt.show()

In [ ]:
d1.head(20)

In [ ]:
ppn = ds.divide_frame(dataf = d1,columns = features+['f1'],teacher_column = "Transported",classifier = 'Tree', criterion_tree = "entropy", depth = 3, random_state_sample = 1,random_state_ppn = 2, fraction = 0.8,plot = False, res = 0.01)